In [2]:
using GLPK
using JuMP
using QPSReader
using NLPModels
using QuadraticModels

In [12]:
function createQuadraticModel(qpdata, name_pb)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0,  name=name_pb)

end


function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function OptimizeGLPK(qpdata; name_pb = "lp_glpk")
    
    model = Model(GLPK.Optimizer)
    QM = createQuadraticModel(qpdata, "QM")
    SM = SlackModel(QM)
    f = grad(SM, zeros(length(SM.meta.x0)))
    Aeq = jac(SM, SM.meta.x0) 
    beq = SM.meta.lcon
    lb = SM.meta.lvar
    ub = SM.meta.uvar
    n = length(f)

    @variable(model, lb[i] <= x[i=1:n] <= ub[i])
    @objective(model, Min, f'*x)
    @constraint(model, con, Aeq*x .== beq)
    
    optimize!(model)
    
    #println(termination_status(model))
    #prontln(primal_status(model))
    #println(dual_status(model))
    println("objective value = " , objective_value(model))
    #println(value(x[1]))
    
end


OptimizeGLPK (generic function with 1 method)

In [13]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes"
afiro = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(afiro);


┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [14]:
OptimizeGLPK(qpdata2)

objective value = -464.75314285714285
